In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
import string
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score

In [ ]:
train = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
train['length'] = train['text'].apply(len)

In [ ]:
train.groupby('target').describe()

In [ ]:
train[train['length'] == 157]['text'].iloc[0]

In [ ]:
train.head()

In [ ]:
test = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [ ]:
test.head()

In [ ]:
test['length'] = test['text'].apply(len)

In [ ]:
test.head()

# Data Visualization

In [ ]:
train['length'].hist(bins=200)

In [ ]:
train.hist(column='length',by='target',bins=200,figsize=(12,6))

In [ ]:
sns.heatmap(train.isnull(),yticklabels=False,cbar=False,cmap='viridis')

In [ ]:
# nltk.download_shell()

# Text Pre-Processing

In [ ]:
string.punctuation

In [ ]:
stopwords.words('english')

In [ ]:
#1. Remove Punctuations
#2. Remove stopwords
#3. return list of clean words

In [ ]:
def text_process(text):
    nopunc = [punc for punc in text if punc not in string.punctuation]
    nopunc = ''.join(nopunc)
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english') ]

In [ ]:
train['text'].head(5).apply(text_process)

In [ ]:
# orginal Dataframe
train.head()

# train_test_split

In [ ]:
X = train['text']
y = train['target']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=101)

In [ ]:
disaster_test = test['text']

# Random Forest

In [ ]:
rfc = Pipeline([
    ('bow',CountVectorizer(analyzer=text_process)),
    ('tfidf',TfidfTransformer()),
    ('classifier',RandomForestClassifier())
])

rfc.fit(X_train,y_train)
predict_rfc = rfc.predict(X_test)

print(confusion_matrix(y_test,predict_rfc))
print(classification_report(y_test,predict_rfc))
print('accuracy_score',accuracy_score(y_test,predict_rfc))
print('f1_score',f1_score(y_test,predict_rfc))

# Naive Bayes

In [ ]:
nb = Pipeline([
    ('bow',CountVectorizer(analyzer= text_process)),
    ('tfidf',TfidfTransformer()),
    ('classifier',MultinomialNB())
])

nb.fit(X_train,y_train)
predict_nb = nb.predict(X_test)

print(confusion_matrix(y_test,predict_nb))
print(classification_report(y_test,predict_nb))
print('accuracy_score',accuracy_score(y_test,predict_nb))
print('f1_score',f1_score(y_test,predict_nb))

# Support Vector Machine

In [ ]:
svm = Pipeline([
    ('bow',CountVectorizer(analyzer = text_process)),
    ('tfidf',TfidfTransformer()),
    ('classifier',SVC())
])

svm.fit(X_train,y_train)
predict_svm = svm.predict(X_test)

print(confusion_matrix(y_test,predict_svm))
print(classification_report(y_test,predict_svm))
print('accuracy_score',accuracy_score(y_test,predict_svm))
print('f1_score',f1_score(y_test,predict_svm))

# Submission

In [ ]:
test.head()

In [ ]:
# submission 1
final_predict = nb.predict(disaster_test)
submission = pd.DataFrame({'id':test.id,'target':final_predict})
submission.to_csv('submission.csv',index=False)